# Tarea 2
A partir del corpus proporcionado realizar un modelo del lenguaje neuronal con base en la arquitectura de word2vec. Siganse los siguientes pasos:

1. Limpiar los Textos y apliccar stemming a las palabras. **[DONE]**
2. Insertar simbolos de inicio y final de cadena. **[DONE]**
3. Obtener los bigramas que aparecen en este texto. **[DONE]**
4. Entrenar con los bigramas la red neuronal y obtener los valores para los hiperparamteros. Tomar de 100 a 300 unidades para la capa oculta.
5. Obtener la matriz A y Pi a partir de las salidas de la red neuronal 
6. Calcular la probabilidad de las siguientes oraciones:
    - Nos bañamos con agua caliente 
    - El animalito le olía la cabeza
    - Pascuala ordeñaba las vacas

In [78]:
# Importamos las librerias

# Usamos os para cargar los libros
from os import listdir,getcwd
from os.path import isfile, join
import re


## Importamos el corpus
Lo primero que hacemos es importar el corpus al notebook para que podamos utilizarlos. En este caso definimos dos formas de cargar el corpus, ya sea por documento o cargando todos los documentos del folder.

In [79]:

# Obtenemos el path del folder donde se almacenan los corpus
folder_path = (getcwd() + r"/CorpusDocs")

# Los almacenamos en una lista donde se almacenan los nombres de los archivos.
# Esto es en caso de que usemos todos los corpus.
corpus_name_list = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]

def loadAllCorpus():
    """
    Esta funcion carga todos los corpus que estan en el folder Corpus Docs.
    """
    corpis = ''
    for file in corpus_name_list:
        with open("./CorpusDocs/" + file, 'r', encoding="utf8") as f:
            corpis += f.read()
    return corpis

def loadCorpus(corpus_name):
    """
    Esta funcion nos sirve para cargar un corpus especifico
    """
    with open("./CorpusDocs/" + corpus_name, 'r', encoding="utf8") as f:
        corpus = f.read()
    return corpus
        

In [80]:
# Cargamos el corpus.

#corpus = loadAllCorpus()
corpus = loadCorpus('corpusML.txt')

## Limpieza del Texto
Separamos las palabras de las oraciones para poder trabajar con ellas individualmente

In [81]:
def add_eos_init(corpus):
    """
    Adds <eos> and <init> to a corpus. Based on line jumps
    """
    
    init = '<init> '
    eos = ' <eos>'
    corpus_in_eo = []


    corpus_in_eo = init + corpus + eos
    corpus_in_eo= corpus_in_eo.replace("\n", eos +" \n"+init)

    return corpus_in_eo

In [82]:
corpus_init = add_eos_init(corpus)
words = corpus_init.split()
print(words[:20])

['<init>', 'Comencé', 'a', 'trabajar', 'y', 'me', 'pegaron,', 'me', 'maltrataron', 'con', 'chicote', '<eos>', '<init>', 'Mis', 'patrones', 'me', 'pegaron', 'porque', 'no', 'me']


Eliminamos la puntuación del documento, acentos y normalizamos el texto en minusculas. Para hacer la eliminación de los símbolos de puntuación utilziamos una tabla de traducción para optimizar la velocidad de procesamiento. Tambien fue necesario extender la tabla de símbolos para que incluyera algunos símbolos latinos que faltaban.

Para eliminar acentos usamos la libreria unidecode que se tiene que instalar adicionalmente: `pip install unidecode`

In [83]:
import string
import unidecode

# Para poder mantener algunos flags quitamos < > de los elementos que se pueden eliminar
# de los simbolos de puntuación
punct = string.punctuation.replace("<", '')
punct = punct.replace(">", '')

print(punct)

!"#$%&'()*+,-./:;=?@[\]^_`{|}~


In [84]:
lat_punctuation = punct+'¿¡1234567890'
#print(lat_punctuation)

table = str.maketrans('', '', lat_punctuation)

In [85]:
clean_words = []

for word in words:
    word = word.lower()               # Minusculas
    word = unidecode.unidecode(word)  # Quitamos acentos.
    
    # Clean punctuation
    temp_w = []
    for letter in word:
        if letter not in lat_punctuation:
            temp_w.append(letter)
        word = ''.join(temp_w)

    clean_words.append(word)


## Stemming de Palabras
Para hacer el stemming de las palabras usamos NLTK. Para esto hay que instalar NLTK: 

Lo primero que hacemos es definir un stemmer. En este caso usaremos [Snowball Stemmer](http://snowball.tartarus.org/texts/introduction.html).

In [86]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

In [87]:
stemmed_text = []
for word in clean_words:
    stemmed_text.append(stemmer.stem(word))
    
print(stemmed_text[:10])

['<init>', 'comenc', 'a', 'trabaj', 'y', 'me', 'peg', 'me', 'maltrat', 'con']


## Bigramas
Para obtener los biogramas existentes del corpus creamos una fucnion que nos sierve para obtener todos los bigramas que existen en el corpus y a traves de el generar una lista de ellos.

In [88]:
def create_ngrams(stemmed_text, n):
    """
    Creates an n-gram structure from a stemmed or tokenized text. 
    
    Params
    ------
    stemmed_text: Tokens or stemmed words of a corpus
    n: the size of the n gram ex. 2 for a bigram
    """
    return zip(*[stemmed_text[i:] for i in range(n)])

In [89]:
bigramas = list(create_ngrams(stemmed_text, 2))

In [90]:
print(bigramas[0])

('<init>', 'comenc')


In [91]:
import collections
counter=collections.Counter(bigramas)       
    

In [92]:
print(counter)

Counter({('<eos>', '<init>'): 1704, ('<init>', 'pues'): 177, ('entonc', '<eos>'): 86, ('<init>', 'se'): 79, ('<init>', 'me'): 77, ('otra', 'vez'): 72, ('<init>', 'y'): 68, ('<init>', 'per'): 65, ('<init>', 'no'): 61, ('<init>', 'entonc'): 49, ('<init>', 'despu'): 47, ('nom', '<eos>'): 44, ('<init>', 'el'): 43, ('<init>', 'lueg'): 43, ('par', 'que'): 38, ('cas', '<eos>'): 35, ('<init>', 'cuand'): 33, ('<init>', 'si'): 31, ('en', 'la'): 30, ('<init>', 'com'): 30, ('<init>', 'este'): 29, ('a', 'la'): 27, ('pues', 'se'): 26, ('pues', 'me'): 26, ('pues', 'entonc'): 26, ('<init>', 'lo'): 26, ('<init>', 'le'): 26, ('<init>', 'por'): 25, ('<init>', 'la'): 25, ('<init>', 'ya'): 25, ('en', 'el'): 25, ('com', '<eos>'): 24, ('per', 'pues'): 23, ('que', 'se'): 21, ('fue', 'a'): 21, ('no', 'se'): 21, ('que', 'no'): 21, ('<init>', 'habi'): 20, ('trabaj', '<eos>'): 20, ('por', 'eso'): 20, ('iba', 'a'): 20, ('<init>', 'he'): 19, ('ya', 'no'): 19, ('con', 'el'): 19, ('pues', 'asi'): 18, ('de', 'la'): 18

In [93]:
print(counter['<init>', 'a'])

6


In [94]:
# Obtenemos el alfabeto de las palabras del stem
#alfabeto = set(stemmed_text)

alfabetoPI = []
for stem in stemmed_text:
    if stem not in alfabetoPI:
        alfabetoPI.append(stem)
        

alfabetoPI.remove('<init>')

print("AlfabetoPI total: {0}".format(len(alfabetoPI)))


AlfabetoPI total: 1216


## Visualización de Matrices
Utilizamos Pandas para visualizar la matriz A y la matriz Pi

In [95]:
import pandas as pd

In [96]:
MatrixA = pd.DataFrame(index=alfabetoPI, columns=alfabetoPI)

In [97]:
for x in alfabetoPI:
    for y in alfabetoPI:
        MatrixA.set_value(y,x, counter[y,x])

In [98]:
MatrixA

,comenc,a,trabaj,y,me,peg,maltrat,con,chicot,<eos>,...,juguetit,entram,cab,bebecit,tabiqu,calent,pajuel,vapor,quemart,cai
comenc,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a,0,0,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
trabaj,0,2,0,3,0,0,0,0,0,20,...,0,0,0,0,0,0,0,0,0,0
y,0,8,1,0,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
me,0,0,0,0,0,15,2,0,0,0,...,0,0,0,0,0,0,0,0,0,1
peg,0,5,0,0,1,0,0,1,0,11,...,0,0,0,0,0,0,0,0,0,0
maltrat,0,0,0,0,0,0,0,1,0,2,...,0,0,0,0,0,0,0,0,0,0
con,0,0,4,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
chicot,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
<eos>,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:

MatrixPI = pd.DataFrame(index=alfabetoPI, columns=['<init>'])

In [100]:
for y in alfabetoPI:
    if y is not '<eos>':
        MatrixPI.set_value(y,'<init>', counter['<init>',y])

In [101]:
MatrixPI

,<init>
comenc,1
a,6
trabaj,2
y,68
me,77
peg,0
maltrat,0
con,9
chicot,0
<eos>,1


# Realizado por:
- Bustos Ramírez Luis Enrique
- Flores Cortés Juan Pablo
